File path to the list of species
```
/home/eeb177-student/Desktop/eeb-177/project/sandbox/WHRN-Harm-PDZD7_comparisson/taxa_list_test_reduced_formated.txt
```

Let's grab the taxa ids of interest

In [1]:
test_list_taxa_file = open("/home/eeb177-student/Desktop/eeb-177/project/sandbox/WHRN-Harm-PDZD7_comparisson/taxa_list_test_reduced_formated.txt", "r").readlines()
test_taxa_list = [] #set up a holder to append into
for line in test_list_taxa_file:
    test_taxa_list.append(line.split(",")[1].strip("\n")) #get the ids only

In [2]:
print(test_taxa_list)

['946362', '192875', '10228', '400682', '9606', '45351', '6087', '7668', '6239', '7227', '37653', '7574']


In [3]:
def parse_and_getID_best_hits(taxa_id_list, file_format_name):
    #  goes through the output of a BLAST xml file 
    #  finds the best result for the taxa's submited
    from Bio.Blast import NCBIXML
    from Bio.SeqRecord import SeqRecord
    #import the required libraries
    
    for taxa_id in taxa_id_list:
        file_name = str("USH_Search_" + "txid" +taxa_id + "[ORGN].xml")
        #this sets the handle, without having to format every single name of the input files
        #added a txid, so we only need to grab only the number from the file, nothing else
        
        result_handle = open(str("/home/eeb177-student/Desktop/eeb-177/project/sandbox/Testrun_multi_genes_same_org/Batch_run_workspace/"+file_name), "r") #sets the result handle to read the results from
        #i've altered this to be able to find the genes without having them in the same directory
        
        blast_records = NCBIXML.parse(result_handle)
        #need to use parse if it has multiple records in it
        for blast_record in blast_records:
            org_desig=file_name.split("_")[2].split("[")[0].replace(" ", "_")
            #properly formats the taxa id so we can use it to name things
            
            homo_sapiens = "[Homo sapiens]"
            blast_query=blast_record.query
            if homo_sapiens in blast_query:
                gene_name=blast_record.query.split("|")[4].split("[")[0].replace(" ", "_").replace("_protein", "").replace("_isoform_b3", "")
                formated_gene_name = gene_name[1:-1]
            else:
                formated_gene_name=blast_query.split("|")[4].split("_")[0]
            #this conditional properly formats the gene name so we can use it to name things
                #Basically there are 2 formats of sequence names I used to do the search
                #this statement switches the naming convention depending on which is used
                #it's not very general, so in the future I will make sure to properly name my search sequences
                #that should make this statement unnecessary 
            
            
            output_file_name=("gene_"+formated_gene_name +"_"+file_format_name+ "_summary.csv")
            #changed this line so we can add the extra variable so we don't have to redefine the function every time we run it
        
            output_file=open(output_file_name, "a")
            
            #this puts together a nicely named file for each gene and species
            
            alignment_file_contents=""
            #setting up an empty list to hold the output of the following loop
            for alignment in blast_record.alignments[:1]:
                #change the number here to determine how many of the hits you the number specifies 
                #if it is 3 that you are grabing the 3 best hits

                score_counter=[]
                e_val_counter=[]

                for hsp in alignment.hsps:
                    evalue=hsp.expect
                    e_val_counter.append(evalue)
                
                seq_designation = alignment.title.split("|")
                gi_number = seq_designation[1]
                ref_number = seq_designation[3]
                annotation = seq_designation[4]
                #breaking apart the annoation of the sequence name
                #otherwise we'd have way too many delimiters within delimiters
                output_line = str(gi_number + ","+ 
                                  ref_number + "," +
                                  annotation+ "," +
                                  str(min(e_val_counter)) + "\n")
                #this sets up the output for each alignment
                #had to alter the line breaks and parenthesis positioning
                
                alignment_file_contents = alignment_file_contents + output_line
                #this writes what we came up with to the holder string
            output_file.write(alignment_file_contents)
            #writing the output to the alignment file
            output_file.close()
        result_handle.close()

In [5]:
parse_and_getID_best_hits(test_taxa_list, "best_hit_reduced_taxa")

## Let's make a nice little script to pull out the relevant GI's

tried to make a bash script work, let's just go with a python script


In [45]:
def get_gi_from_summary(input_file_name):
    #let's format the file name and the contents
    
    intermediate_name = input_file_name.split("_")[1:-1]
    #gets rid of uneccessary parts of the name
    final_name = "seqs_" + "_".join(intermediate_name)+ "_gi.txt"
    #this gives us the formated output name
    
    output_file = open(final_name, "w")
    #open the file to write to it
    
    
    input_file_contents = open(input_file_name, "r").read()
    for line in input_file_contents.split("\n")[:-1]:
    # the -1 gets around the extra line at the end of the file 
    #so everything is ready to input for the next process
        output_file.write(str(line.split(",")[0] + "\n"))
    output_file.close()
    # 
    #    for line in input_file_contents.split("\n"):
#        output_file_contents = print(line)
#    print(output_file_contents)

In [46]:
get_gi_from_summary("gene_cadherin_23_best_hit_reduced_taxa_summary.csv")

In [47]:
gene_list =("gene_cadherin_23_best_hit_reduced_taxa_summary.csv",
            "gene_CIB2_best_hit_reduced_taxa_summary.csv",
            "gene_Clarin_1_best_hit_reduced_taxa_summary.csv",
            "gene_GPR98_best_hit_reduced_taxa_summary.csv",
            "gene_GPR98_best_hit_reduced_taxa_summary.csv",
            "gene_harmonin_best_hit_reduced_taxa_summary.csv",
            "gene_myosin_VIIA_best_hit_reduced_taxa_summary.csv",
            "gene_PDZD7_best_hit_reduced_taxa_summary.csv",
            "gene_protocadherin_15_best_hit_reduced_taxa_summary.csv",
            "gene_USH1G_best_hit_reduced_taxa_summary.csv",
            "gene_USH2A_best_hit_reduced_taxa_summary.csv",
            "gene_WHRN_best_hit_reduced_taxa_summary.csv")

In [49]:
for gene in gene_list:
    get_gi_from_summary(gene)

Switch to the shell
going to use grep to grab all the things with _gi in the name

```
ls | grep "_gi.txt" > organization_best_hits_gi_filenames.txt
```

In [54]:
gi_files_to_download = open("organization_best_hits_gi_filenames.txt", "r").readlines()
print(gi_files_to_download)

['seqs_cadherin_23_best_hit_reduced_taxa_gi.txt\n', 'seqs_CIB2_best_hit_reduced_taxa_gi.txt\n', 'seqs_Clarin_1_best_hit_reduced_taxa_gi.txt\n', 'seqs_GPR98_best_hit_reduced_taxa_gi.txt\n', 'seqs_harmonin_best_hit_reduced_taxa_gi.txt\n', 'seqs_myosin_VIIA_best_hit_reduced_taxa_gi.txt\n', 'seqs_PDZD7_best_hit_reduced_taxa_gi.txt\n', 'seqs_protocadherin_15_best_hit_reduced_taxa_gi.txt\n', 'seqs_USH1G_best_hit_reduced_taxa_gi.txt\n', 'seqs_USH2A_best_hit_reduced_taxa_gi.txt\n', 'seqs_WHRN_best_hit_reduced_taxa_gi.txt\n']


## Ok, well that is working after a lot of work

In [55]:
def download_seqs_from_list_and_autoname(in_filename):    
    with open(in_filename, "r") as query_file:
        query_ids = query_file.read().splitlines()
    #open the file, grab the ids

    from Bio import Entrez
    Entrez.email = "hspeck@ucla.edu"
    # import Bio Entrez, let NCBI know who I am

    search_results = Entrez.read(Entrez.epost(db ="protein", 
                                              id = ",".join(query_ids)))  # needs the id's to be as a list separated by commas
    #upload the IDs to NCBI
    webenv_return = search_results["WebEnv"]
    query_key_return = search_results["QueryKey"]
    #grab the relevant variables to call on the stuff we posted

    count = len(query_ids)
    #assigns the count based on the number of sequences we searched for

    from urllib.error import HTTPError
    # load required library for the try and except conditions

    batch_size = 20
    #this determines how many things we retrieve and write to the file
    #can safely be larger in the future

    out_filename = str(in_filename[:-7]+".fasta")
    #let's alter the file name a bit so we can easily move on
    
    #attempting to rename the file based on the input of the original file
    out_handle = open(out_filename, "w")
    #open file to write to

    for start in range(0, count, batch_size):
        end = min(count, start+batch_size)
        print("Going to download record %i to %i" % (start+1, end))
        attempt = 0
        while attempt < 3:
            attempt += 1
            try:
                fetch_handle = Entrez.efetch(db="protein", #says which db
                                             rettype="fasta", #says what format the data should be in
                                             retmode="text", #what the output should be
                                             retstart = start, #say what range of results you want returned
                                             retmax = batch_size, #say end of range of results want returned
                                             webenv = webenv_return, #specify the info we uploaded with ePost
                                             query_key = query_key_return)
            except HTTPError as err:
                if 500 <= err.code <= 599:
                    print("Recieved error from server %s" % err)
                    print("Attempt %i of 3" % attempt)
                    time.sleep
                else:
                    raise
        data = fetch_handle.read()
        fetch_handle.close()
        out_handle.write(data)
    out_handle.close()

In [52]:
for file in gi_files_to_download:
    print(file.replace("\n", ""))
    # this is needed to actually input the right thingy

organization_best_hits_gi.txt
seqs_cadherin_23_best_hit_reduced_taxa_gi.txt
seqs_CIB2_best_hit_reduced_taxa_gi.txt
seqs_Clarin_1_best_hit_reduced_taxa_gi.txt
seqs_GPR98_best_hit_reduced_taxa_gi.txt
seqs_harmonin_best_hit_reduced_taxa_gi.txt
seqs_myosin_VIIA_best_hit_reduced_taxa_gi.txt
seqs_PDZD7_best_hit_reduced_taxa_gi.txt
seqs_protocadherin_15_best_hit_reduced_taxa_gi.txt
seqs_USH1G_best_hit_reduced_taxa_gi.txt
seqs_USH2A_best_hit_reduced_taxa_gi.txt
seqs_WHRN_best_hit_reduced_taxa_gi.txt


In [61]:
#testing the rename function

for file_name in gi_files_to_download:
    intermediate_name = file_name.split("_")[1:-1]
    muscle_align = "muscle_align_" + "_".join(intermediate_name) + ".txt"
    print(muscle_align)

muscle_align_cadherin_23_best_hit_reduced_taxa.txt
muscle_align_CIB2_best_hit_reduced_taxa.txt
muscle_align_Clarin_1_best_hit_reduced_taxa.txt
muscle_align_GPR98_best_hit_reduced_taxa.txt
muscle_align_harmonin_best_hit_reduced_taxa.txt
muscle_align_myosin_VIIA_best_hit_reduced_taxa.txt
muscle_align_PDZD7_best_hit_reduced_taxa.txt
muscle_align_protocadherin_15_best_hit_reduced_taxa.txt
muscle_align_USH1G_best_hit_reduced_taxa.txt
muscle_align_USH2A_best_hit_reduced_taxa.txt
muscle_align_WHRN_best_hit_reduced_taxa.txt


Let's make it download and run MUSCLE automatically

so we need to incorporate the muscle step into the download

preferably wed rename the files at the same time


In [64]:
def download_seqs_from_list_autoname_align(in_filename):    
    with open(in_filename, "r") as query_file:
        query_ids = query_file.read().splitlines()
    #open the file, grab the ids

    from Bio import Entrez
    Entrez.email = "hspeck@ucla.edu"
    # import Bio Entrez, let NCBI know who I am

    search_results = Entrez.read(Entrez.epost(db ="protein", 
                                              id = ",".join(query_ids)))  # needs the id's to be as a list separated by commas
    #upload the IDs to NCBI
    webenv_return = search_results["WebEnv"]
    query_key_return = search_results["QueryKey"]
    #grab the relevant variables to call on the stuff we posted

    count = len(query_ids)
    #assigns the count based on the number of sequences we searched for

    from urllib.error import HTTPError
    # load required library for the try and except conditions

    batch_size = 20
    #this determines how many things we retrieve and write to the file
    #can safely be larger in the future

    out_filename = str(in_filename[:-7]+".fasta")
    #let's alter the file name a bit so we can easily move on
    
    #attempting to rename the file based on the input of the original file
    out_handle = open(out_filename, "w")
    #open file to write to

    for start in range(0, count, batch_size):
        end = min(count, start+batch_size)
        print("Going to download record %i to %i" % (start+1, end))
        attempt = 0
        while attempt < 3:
            attempt += 1
            try:
                fetch_handle = Entrez.efetch(db="protein", #says which db
                                             rettype="fasta", #says what format the data should be in
                                             retmode="text", #what the output should be
                                             retstart = start, #say what range of results you want returned
                                             retmax = batch_size, #say end of range of results want returned
                                             webenv = webenv_return, #specify the info we uploaded with ePost
                                             query_key = query_key_return)
            except HTTPError as err:
                if 500 <= err.code <= 599:
                    print("Recieved error from server %s" % err)
                    print("Attempt %i of 3" % attempt)
                    time.sleep
                else:
                    raise
        data = fetch_handle.read()
        fetch_handle.close()
        out_handle.write(data)
    out_handle.close()
    
    
    
    #now to run MUSCLE on it
    raw_seq_input = out_filename
    intermediate_name = in_filename.split("_")[1:-1] 
    #grabs the core of the name
    muscle_align_name = "muscle_align_" + "_".join(intermediate_name) + ".txt" 
    # generates a named file for the muscle alginment output
    
    from Bio.Align.Applications import MuscleCommandline
    #imports the relevant biopython module for python
    muscle_cline = MuscleCommandline(input = raw_seq_input, 
                                 out = muscle_align_name)
    stdout, stder = muscle_cline()

## NOW TO run the function

In [65]:
for file in gi_files_to_download:
    formated_input = file.replace("\n", "")
    download_seqs_from_list_autoname_align(formated_input)

Going to download record 1 to 12
Going to download record 1 to 12
Going to download record 1 to 5
Going to download record 1 to 12
Going to download record 1 to 12
Going to download record 1 to 12
Going to download record 1 to 12
Going to download record 1 to 12
Going to download record 1 to 12
Going to download record 1 to 12
Going to download record 1 to 12
